# Sleep Stage Classification using EEG and IMU Data

### Authors: Eidan Tzdaka, David Trocellier

## Import necessary libraries

In [ ]:
import h5py  # Read and write HDF5 files from Python
import matplotlib.pyplot as plt  # For plotting graphs
import numpy as np  # For numerical computations
import pandas as pd  # For data processing (CSV, DataFrames)
import tensorflow as tf  # TensorFlow for deep learning models
from keras import Input, Model  # Model building components
from scipy.signal import stft  # Short-time Fourier Transform for signal processing
from sklearn.metrics import confusion_matrix, classification_report  # For evaluating model performance
from sklearn.model_selection import KFold  # K-Fold cross-validation
from tensorflow.keras.callbacks import EarlyStopping  # Callback for stopping training early
from tensorflow.keras.layers import (Lambda, BatchNormalization, MaxPooling1D, 
                                     Dense, Conv1D, Dropout, GRU, 
                                     TimeDistributed, GlobalAveragePooling1D)
from tensorflow.keras.optimizers import Adam  # Optimizer for model training
import os

## Section 1: Load Data Files

In [ ]:
data_path = os.getcwd()  # Get the current working directory
file_xtrain = os.path.join(data_path, 'X_train.h5')  # Path to training data (EEG/IMU signals)
file_xtest = os.path.join(data_path, 'X_test.h5')  # Path to testing data
file_ytrain = os.path.join(data_path, 'y_train.csv')  # Path to labels for training data

## Section 2: Data Preprocessing Functions

In [ ]:
# Function to normalize EEG data between -150 and 150
def normalize_data(eeg_array):
    """Normalize signal between 0 and 1."""
    normalized_array = np.clip(eeg_array, -150, 150)
    normalized_array = normalized_array / 150
    return normalized_array

# Function to apply Short-Time Fourier Transform (STFT) for data preprocessing
def stft_preprocessing(data, mean=None, var=None):
    """Apply STFT and perform mean-variance normalization."""
    zxx = stft(data, fs=50, nperseg=128, nfft=128, noverlap=64, axis=1)[2]
    zxx = np.log(np.abs(zxx))
    cliped = np.clip(zxx, -20, 20)
    cliped = np.swapaxes(cliped, 1, 2)
    cliped = np.swapaxes(cliped, 2, 3)

    newdata = cliped
    print(newdata.shape)
    
    if mean is None:
        mean = newdata.mean()
    newdata = newdata - mean
    
    if var is None:
        var = newdata.var()
    newdata = newdata / var

    return newdata, mean, var

# Function to create positional embeddings
def positional_embeding(data):
    """Compute embedded position using cosine functions."""
    data = np.array(data)
    pos = np.zeros([data.shape[0], 6])
    pos[:, 0] = data / 1200  # Normalize data

    angle = [30, 60, 90, 120, 150]
    for i in range(5):
        pos[:, i + 1] = np.cos((data * np.pi) / angle[i])
    
    return pos

# Function to prepare and split the data into trainable components
def split_data(input_list):
    """Prepare training data and return EEG raw, STFT, and embedded position data."""
    with h5py.File(file_xtrain, "r") as fi:
        if len(input_list) == 1:
            x_data = fi[input_list[0]][()]
        else:
            x_data = np.zeros([24688, 1500, len(input_list)], dtype=np.float64)
            for i in range(0, len(input_list)):
                if 'x' in input_list[i] or 'y' in input_list[i] or 'z' in input_list[i]:
                    f1 = interp1d(np.arange(0, 300), fi[input_list[i]][()], axis=1)
                    xnew = np.linspace(0, 30, num=1500)
                    x_data[0:24688, 0:1500, i] = f1(xnew)
                else:
                    x_data[0:24688, 0:1500, i] = fi[input_list[i]][()]
        
        # Reading metadata and labels
        y_data_org = pd.read_csv(file_ytrain)['sleep_stage'].to_numpy()
        metadata = fi["index_window"][()]

        # Masking EEG channels and normalizing
        mask_eeg = [0, 1, 2, 3]
        x_eeg_raw_data = normalize_data(x_data[:, :, mask_eeg])
        x_stft_data = np.copy(x_eeg_raw_data)

        # Drop some channels to avoid overfitting
        for subjects in range(x_stft_data.shape[0]):
            for channels in range(x_stft_data.shape[2]):
                if np.random.uniform() < 0.1:
                    x_stft_data[subjects, :, channels] *= 0

        # Perform STFT preprocessing
        x_stft_data, mean_eeg, var_eeg = stft_preprocessing(x_stft_data)
        
        # Generate positional embeddings
        x_emb_pos_data = positional_embeding(metadata)
        x_emb_pos_data = np.append(x_emb_pos_data[0:-1, :], x_emb_pos_data[1:, :], axis=1)
        x_emb_pos_data = x_emb_pos_data.reshape(24687, 2, 6)
        
        # Reshape STFT data
        x_stft_data = np.append(x_stft_data[0:-1, :, :, :], x_stft_data[1:, :, :, :], axis=1)
        x_stft_data = x_stft_data.reshape(24687, 2, 4, 25, 65)
        
        # Update EEG raw data
        x_eeg_raw_data = np.append(x_eeg_raw_data[0:-1, :, :], x_eeg_raw_data[1:, :, :], axis=1)

        y_data_org = y_data_org[1:]

        print('Data preparation complete.')
    
    return x_eeg_raw_data, x_stft_data, x_emb_pos_data, y_data_org



## Section 3: Preprocessing the Data 

In [ ]:
# Input signal list for training
input_signals_list = ['eeg_4', 'eeg_5', 'eeg_6', 'eeg_7', 'x', 'y', 'z']

# Prepare training data
x_eeg_raw, x_stft, x_emb_pos, y_data = split_data(input_signals_list)

# Check data shapes to ensure they are consistent
print('y_data.shape = ', y_data.shape)
print('x_eeg.shape = ', x_stft.shape)
print('x_eeg_normalized.shape = ', x_eeg_raw.shape)
print('x_meta.shape = ', x_emb_pos.shape)

shapes = np.array([y_data.shape[0], x_stft.shape[0], x_eeg_raw.shape[0], x_emb_pos.shape[0]])
if np.sum(np.abs(np.diff(shapes))) != 0:
    print("Shape error in data alignment.")

## Section 3: Create the Model

In [ ]:
# Custom loss function
def custom_loss(ytrue, ypred):
    """SparseCategoricalCrossentropy with class weights."""
    scce = tf.keras.losses.SparseCategoricalCrossentropy()
    weight = tf.constant([[0.94355903, 2.70683738, 0.96141876, 0.55233263, 0.62986756]])
    
    new_y = tf.expand_dims(ypred, axis=1)
    new_weight = tf.matmul(weight, new_y, transpose_b=True)
    score = scce(ytrue, ypred, sample_weight=new_weight)

    return score

In [ ]:
# Optimizer setup
optimizer = Adam(learning_rate=0.001)
# Cross-validation setup
kfold = KFold(n_splits=30, shuffle=True)

# Attention Layer class
class Attention(Layer):
    """Custom attention layer based on Dreem open repository."""
    
    def __init__(self, input_dim, context_size=25):
        super(Attention, self).__init__()  
        init = tf.initializers.GlorotUniform()
        self.context_matrix = tf.Variable(init(shape=(context_size, input_dim)), trainable=True)
        self.context_bias = tf.Variable(init(shape=(1, context_size)), trainable=True)
        self.context_vector = tf.Variable(init(shape=(context_size, 1)), trainable=True)

    def get_config(self):
        config = super().get_config()
        config.update({
            'context_matrix': self.context_matrix,
            'context_bias': self.context_bias,
            'context_vector': self.context_vector
        })
        return config

    def call(self, x, **kwargs):
        """Compute the attention weights and apply them to the input."""
        batch_size, length, n_features = x.shape
        x_att = tf.reshape(x, [-1, n_features])
        u = tf.linalg.matmul(x_att, tf.transpose(self.context_matrix, perm=[1, 0])) + self.context_bias
        u = tf.nn.tanh(u)
        uv = tf.linalg.matmul(u, self.context_vector)
        uv = tf.reshape(uv, [-1, length])
        alpha = tf.nn.softmax(uv, axis=1)
        alpha = tf.expand_dims(alpha, axis=-1)
        x_out = alpha * x
        x_out = tf.math.reduce_sum(x_out, axis=1)

        return x_out
    

# Function for creating K-fold validation for unique subjects
def custom_Kfold(n):
    """Ensure training and validation happen on different subjects."""
    with h5py.File(file_xtrain, "r") as fi:
        x_metadata = fi['index'][()]
    
    x_metadata = x_metadata[1:]
    indices = []
    
    for i in range(n):
        train_indices, val_indices = np.argwhere(x_metadata % n != 0 + i), np.argwhere(x_metadata % n == 0 + i)
        np.random.shuffle(train_indices)
        np.random.shuffle(val_indices)
        indices.append({"train": np.squeeze(train_indices), "validation": np.squeeze(val_indices)})

    return indices

# Reshaping function for backend usage
def backend_reshape(x):
    """Reshape based on batch size and channels."""
    return tf.keras.backend.reshape(x, (-1, n_channels, spectrogram_length, filter_size))


In [ ]:
# Initialize hidden state for GRU and set fold number for cross-validation
hidden = None  # Initial hidden state for GRU layers, will be updated during training
fold_no = 1    # Counter to keep track of which fold is being processed in K-fold cross-validation

# Model configuration parameters

# Filter and channel reduction settings
C = 4                # The number of channels to reduce to (channel reduction)
filter_reduc = 40     # The number of filters to reduce in the first dense layer

# Attention layer context size
att_context = 20      # Size of the context for the attention mechanism

# GRU (Gated Recurrent Unit) configuration
m1 = 128  # Number of units in the first GRU layer
m2 = 128  # Number of units in the second GRU layer

# Dropout rates to prevent overfitting
p1 = 0.2  # Dropout rate for the first GRU layer and channel reduction layers
p2 = 0.2  # Dropout rate for the second GRU layer and other fully connected layers

# Extract shape information from the input STFT data
_, temporal_context, n_channels, spectrogram_length, filter_size = x_stft.shape  # Shape of the STFT data
number_channels = n_channels  # Number of channels in the EEG data

# Define inputs for the model
inputA = Input(shape=(temporal_context, n_channels, spectrogram_length, filter_size))  # Input for STFT features
inputC = Input(shape=(2, 6))  # Input for positional embeddings (metadata)
inputD = Input(shape=134)  # Placeholder input, might be used for additional features (unused in this code)
inputH = Input(shape=(3000, number_channels))  # Input for raw EEG data (used in the CNN)

# Processing path for STFT data

# Reshape inputA to adjust the dimensions for backend processing
k = Lambda(backend_reshape)(inputA)

# Apply filter reduction using TimeDistributed Dense layer
k = TimeDistributed(Dense(filter_reduc))(k)  # Reduce the number of filters (40 in this case)
k = tf.transpose(k, perm=[0, 3, 2, 1])  # Transpose to adjust the shape for the next operations

# Apply channel reduction using TimeDistributed Dense layer
k = TimeDistributed(Dense(C))(k)  # Reduce the number of channels (to 4 in this case)
k = tf.transpose(k, perm=[0, 3, 2, 1])  # Transpose to maintain the correct shape

# Apply dropout to reduce overfitting
k = tf.keras.layers.Dropout(p1)(k)

# Reshape and transpose for GRU layers
k = tf.keras.layers.Reshape((-1, spectrogram_length))(k)  # Flatten for feeding into the GRU layer
k = tf.transpose(k, perm=[0, 2, 1])  # Adjust dimensions for time steps

# Apply bidirectional GRU layer with 128 units
k = tf.keras.layers.Bidirectional(GRU(m1, return_sequences=True, go_backwards=False))(k)

# Apply dropout again to prevent overfitting
k = tf.keras.layers.Dropout(p1)(k)

# Attention mechanism to focus on important features
k = Attention(m1 * 2, att_context)(k)  # Apply attention with a context size of 20
k = tf.reshape(k, [-1, temporal_context, 2 * m1])  # Reshape the output for further processing

# Define the model for STFT data
model_A = Model(inputs=inputA, outputs=k)

# Processing path for positional embedding data (inputC)

# Feed the positional embedding input through a series of dense layers
m = Dense(6, activation=tf.nn.relu)(inputC)
m = Dense(6, activation=tf.nn.relu)(m)
m = Dense(6, activation=tf.nn.relu)(m)
m = Dense(6, activation=tf.nn.relu)(m)
m = Dense(6, activation=tf.nn.relu)(m)

# Define the model for positional embedding
model_C = Model(inputs=inputC, outputs=m)

# Combining the outputs of model_A (STFT features) and model_C (positional embeddings)
combined_input = concatenate([model_A.output, model_C.output])  # Concatenate the two outputs
combined = tf.keras.layers.Reshape((combined_input.shape[1], -1))(combined_input)  # Reshape combined output

# Apply a dense layer to the combined output for residual connection
x_residual = Dense(m1 * 2)(combined)

# Apply bidirectional GRU layer on the combined input with a residual connection
x_lstm = tf.keras.layers.Bidirectional(GRU(m2, return_state=True, return_sequences=True))(combined,
                                                                                          initial_state=hidden)  # First GRU
hidden = x_lstm[1:]  # Update hidden state
x_lstm = x_lstm[0]  # Extract GRU output
x_cat = (x_residual + x_lstm) / 2  # Combine residual connection with GRU output
combined = Dropout(p2)(x_cat)  # Apply dropout

# Second layer with bidirectional GRU and residual connection
x_residual = Dense(m1 * 2)(combined)
x_lstm = tf.keras.layers.Bidirectional(GRU(m2, return_state=True, return_sequences=True))(combined,
                                                                                          initial_state=hidden)  # Second GRU
hidden = x_lstm[1:]  # Update hidden state
x_lstm = x_lstm[0]
x_cat = (x_residual + x_lstm) / 2  # Combine residual connection with GRU output
combined = Dropout(p2)(x_cat)

# Apply global average pooling to reduce the dimensionality
combined = GlobalAveragePooling1D()(combined)

# Define the combined model for STFT and positional embedding data
combined = Model(inputs=[model_A.input, model_C.input], outputs=combined)

# Processing path for raw EEG data using a 1D CNN

# Normalize raw EEG data using batch normalization
raws = BatchNormalization()(inputH)

# Apply multiple Conv1D and MaxPooling1D layers to extract features from raw EEG data
raws = Conv1D(filters=60, kernel_size=20, padding='SAME', activation=tf.nn.leaky_relu)(raws)
raws = MaxPooling1D(pool_size=6)(raws)
raws = Conv1D(filters=80, kernel_size=15, padding='SAME', activation=tf.nn.leaky_relu)(raws)
raws = MaxPooling1D(pool_size=6)(raws)
raws = Dropout(0.25)(raws)
raws = Conv1D(filters=80, kernel_size=10, padding='SAME', activation=tf.nn.leaky_relu)(raws)
raws = MaxPooling1D(pool_size=6)(raws)
raws = Conv1D(filters=80, kernel_size=10, padding='SAME', activation=tf.nn.leaky_relu)(raws)
raws = MaxPooling1D(pool_size=4)(raws)

# Apply a dense layer to the feature-extracted data
raws = Dense(128, activation=tf.nn.leaky_relu)(raws)

# Apply global average pooling to reduce the dimensionality
z = GlobalAveragePooling1D()(raws)

# Define the model for raw EEG data
z = Model(inputs=inputH, outputs=z)

# Final model combining the outputs from the combined (STFT + positional embedding) and raw EEG CNN paths
last_layer = concatenate([combined.output, z.output])

# Apply a final dense layer with softmax activation for classification
v = Dense(5, activation="softmax")(last_layer)

# Define the full model with all inputs and the final output
model = Model(inputs=[model_A.input, model_C.input, z.input], outputs=v)

# Print the model summary to view the architecture
print(model.summary())

## Section 4: Train the Model

In [ ]:
# Initialize the history variable to store training information
history = None

# Generate subject-wise K-fold cross-validation splits
dic_kfold = custom_Kfold(20)  # Create K-fold splits ensuring different subjects in train/validation sets

# Run the model with K-fold cross-validation
for dic in dic_kfold:
    # Uncomment the line below if using K-fold splits manually
    # train, test = dic.values()

    # Check if it's the 15th fold for training
    if fold_no == 15:
        print(f'Training for fold {fold_no} ...')

        # Compile the model with custom loss function and Adam optimizer
        model.compile(loss=custom_loss,
                      optimizer=optimizer,
                      metrics=['accuracy'])

        # Train the model with the training data and validate on the test data
        history = model.fit([x_stft[train], x_emb_pos[train], x_eeg_raw[train]], y_data[train],
                            batch_size=32,  # Batch size for training
                            validation_data=([x_stft[test], x_emb_pos[test], x_eeg_raw[test]], y_data[test]),  # Validation data
                            callbacks=[EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=15)],  # Early stopping to prevent overfitting
                            epochs=7,  # Train for 7 epochs
                            verbose=1)  # Verbose output to track training progress

        # Print completion message after training for the fold
        print('------------------------------------------------------------------------')
        print(f'DONE training for fold {fold_no} ...')

    # Increase fold number for the next iteration
    fold_no += 1


## Section 5: Evaluate the Model

In [ ]:
if history is not None:
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy with coustom loss')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss with coustom loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (N, M).
    row_labels
        A list or array of length N with the labels for the rows.
    col_labels
        A list or array of length M with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    for edge, spine in ax.spines.items():
        spine.set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=["black", "white"],
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A list or array of two color specifications.  The first is used for
        values below a threshold, the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

def plot_heatmap(y_true,y_pred): 
    
    mat = confusion_matrix(y_true,y_pred)
    mat = mat/np.sum(mat ,axis =0)*100
    mat = mat.astype(np.int)

    fig, ax = plt.subplots()

    im, cbar = heatmap(mat, ['Wake','NREM1','NREM2','NREM3','REM'],['Wake','NREM1','NREM2','NREM3','REM'], ax=ax,
                       cmap="YlGn", cbarlabel="Pourcentage de classification")
    texts = annotate_heatmap(im, valfmt="{x:} %")

    fig.tight_layout()
    plt.show()

y_estimate = model.predict([x_stft[test], x_emb_pos[test], x_eeg_raw[test]])
y_estimate = np.argmax(y_estimate, axis=1)
print('confusion matrix with coustom loss')
plot_heatmap(y_data[test], y_estimate)
print('____________________________________________')
print('')
print('')
print('classification report with coustom loss')
print('')
target_names=['Wake','NREM1','NREM2','NREM3','REM']
# print(confusion_matrix(y_data[test], y_estimate))
print(classification_report(y_data[test], y_estimate, target_names=target_names))


## Section 6: Predict the Test Data

In [ ]:
def test_data(input_list):
    '''get the test data the same as the traning data'''
    with h5py.File(file_xtest, "r") as fi:
        index = fi['index_absolute'][()]
        metadata = fi["index_window"][()]
        if len(input_list) == 1:
            x_data = fi[input_list[0]][()]
        else:
            x_data = np.zeros([24980, 1500, len(input_list)], dtype=np.float64)
            for i in range(0, len(input_list)):
                if 'x' in input_list[i] or 'y' in input_list[i] or 'z' in input_list[i]:
                    f1 = interp1d(np.arange(0, 300), fi[input_list[i]][()], axis=1)
                    xnew = np.linspace(0, 30, num=1500)
                    x_data[0:24980, 0:1500, i] = f1(xnew)
                else:
                    x_data[0:24980, 0:1500, i] = fi[input_list[i]][()]

    # EEG extraction
    mask_eeg = [0, 1, 2, 3]

    x_eeg_normalized = normalize_data(x_data[:, :, mask_eeg])
    x_eeg = np.copy(x_eeg_normalized)
    x_eeg, mean_eeg, var_eeg = stft_preprocessing(x_eeg)
    x_meta = positional_embeding(metadata)
    x_meta = np.append(x_meta[0:-1, :], x_meta[1:, :], axis=1)
    x_meta = x_meta.reshape(24979, 2, 6)
    x_eeg = np.append(x_eeg[0:-1, :, :, :], x_eeg[1:, :, :, :], axis=1)
    x_eeg = x_eeg.reshape(24979, 2, 4, 25, 65)
    x_eeg_normalized = np.append(x_eeg_normalized[0:-1, :, :], x_eeg_normalized[1:, :, :], axis=1)

    print('data ready')

    return x_eeg_normalized, x_eeg, x_meta, index


x_eeg_raw_test, x_stft_test, x_emb_pos_test, index = test_data(input_signals_list)
# predict the test data and save it as CSV
y_test = model.predict([x_stft_test, x_emb_pos_test, x_eeg_raw_test])
y_test = np.argmax(y_test, axis=1)
y_test = np.append(y_test[0], y_test)
if index.shape == y_test.shape:
    df = pd.DataFrame(data={'index': index, 'sleep_stage': y_test})
    df.to_csv('model_num_' + str(fold_no) + '_our_result_late.csv', index=False)
    print('you can save the results')
else:
    print('there is an error in the shape of y_test')